## Load modules and data

In [1]:
#Base
import sys, os
import numpy as np
import xarray as xr
import pandas as pd
import scipy.stats as st
from tqdm.notebook import trange, tqdm

#Plot
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import matplotlib.gridspec as gridspec

#V1DD
from allen_v1dd.client import OPhysClient, OPhysSession
from allen_v1dd.stimulus_analysis import StimulusAnalysis
from allen_v1dd.stimulus_analysis.correlation import calculate_correlation, ProgressBar
from allen_v1dd.stimulus_analysis.natural_scenes import NaturalScenes
from allen_v1dd.stimulus_analysis.drifting_gratings import DriftingGratings

%matplotlib inline
%load_ext autoreload
%autoreload 2

PlotDir = '/home/david.wyrick/projects/V1DD/plots'

In [2]:
## Load data
base_folder = r"\\allen\programs\mindscope\workgroups\surround\v1dd_in_vivo_new_segmentation\data" #Server
base_folder = "/data/v1dd_in_vivo_new_segmentation" # Local on robinson for golden mouse
client = OPhysClient(base_folder)

## Run on all sessions

In [4]:
mouse = 409828 # selected for EM
column = 1

session_ids = client.get_all_session_ids()
# session_ids = [s for s in session_ids if s.startswith(f"M{mouse}") and s[8] == str(column) and s[9] == "3"]
session_ids = [s for s in session_ids if s.startswith(f"M{mouse}") and s[8] == str(column)]

print("Sessions to load:")
print(session_ids)

Sessions to load:
['M409828_11', 'M409828_12', 'M409828_13', 'M409828_14', 'M409828_15', 'M409828_16', 'M409828_17', 'M409828_18', 'M409828_19', 'M409828_1a', 'M409828_1b', 'M409828_1c', 'M409828_1d', 'M409828_1e', 'M409828_1f']


In [5]:
sessions = []

for session_id in tqdm(session_ids):
    session = client.load_ophys_session(session_id=session_id)
    sessions.append(session)

print(f"Loaded {len(sessions)} ophys sessions")

  0%|          | 0/15 [00:00<?, ?it/s]

Loaded 15 ophys sessions


In [ ]:
ns12_cascade = NaturalScenes(sess, 3, ns_type="natural_images_12",trace_type="cascade")

#Calculate metrics 
ns12_cascade_metrics = ns12_cascade.metrics
ns12_cascade_metrics = ns12_cascade_metrics[mask]
ns12_cascade_metrics

In [10]:
plane_info = [] # holds (mouse, column, volume, plane)
ns12 = []
ns118 = []

with tqdm(desc="Loading natural scene analyses", total=sum(len(s.get_planes()) for s in sessions)) as pbar:
    for session in sessions:
        for plane in session.get_planes():
            plane_info.append((session.get_mouse_id(), session.get_column_id(), session.get_volume_id(), plane))


            ns12.append(NaturalScenes(session, plane, ns_type="natural_images_12"))
            ns118.append(NaturalScenes(session, plane, ns_type="natural_images"))
            pbar.update()

Loading natural scene analyses:   0%|          | 0/40 [00:00<?, ?it/s]

In [12]:
RECOMPUTE_METRICS = True
DF_SAVE_DIR = r"/home/david.wyrick/Git/allen_v1dd/dataframes"

if RECOMPUTE_METRICS:

    # Concatenate across the different planes
    print("Concatenating ns12 metrics...")
    ns12_metrics = StimulusAnalysis.concat_metrics(ns12)
 
    print("Merging ns118 metrics...")
    ns118_metrics = StimulusAnalysis.concat_metrics(ns118)
 
    print("Done.", len(ns118_metrics), "total ROIs.")
    ns12_metrics.to_csv(os.path.join(DF_SAVE_DIR, "ns12_metrics_col1.csv"))
    ns118_metrics.to_csv(os.path.join(DF_SAVE_DIR, "ns118_metrics_col1.csv"))

ns12_metrics

Concatenating ns12 metrics...
